In [0]:
!pip install tqdm

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
CS224N 2019-20: Homework 3
parser_transitions.py: Algorithms for completing partial parsess.
Sahil Chopra <schopra8@stanford.edu>
Haoshen Hong <haoshen@stanford.edu>
"""

import sys

class PartialParse(object):
    def __init__(self, sentence):
        """Initializes this partial parse.

        @param sentence (list of str): The sentence to be parsed as a list of words.
                                        Your code should not modify the sentence.
        """
        # The sentence being parsed is kept for bookkeeping purposes. Do not alter it in your code.
        self.sentence = sentence

        ### YOUR CODE HERE (3 Lines)
        ### Your code should initialize the following fields:
        ###     self.stack: The current stack represented as a list with the top of the stack as the
        ###                 last element of the list.
        ###     self.buffer: The current buffer represented as a list with the first item on the
        ###                  buffer as the first item of the list
        ###     self.dependencies: The list of dependencies produced so far. Represented as a list of
        ###             tuples where each tuple is of the form (head, dependent).
        ###             Order for this list doesn't matter.
        ###
        ### Note: The root token should be represented with the string "ROOT"
        ###
        self.stack = ["ROOT"]
        self.buffer = list(sentence)
        self.dependencies = []

        ### END YOUR CODE


    def parse_step(self, transition):
        """Performs a single parse step by applying the given transition to this partial parse

        @param transition (str): A string that equals "S", "LA", or "RA" representing the shift,
                                left-arc, and right-arc transitions. You can assume the provided
                                transition is a legal transition.
        """
        ### YOUR CODE HERE (~7-10 Lines)
        ### TODO:
        ###     Implement a single parsing step, i.e. the logic for the following as
        ###     described in the pdf handout:
        ###         1. Shift
        ###         2. Left Arc
        ###         3. Right Arc
        if transition == "S":
        # SHIFT : POP FIRST OF BUFFER AND INSERT IN STACK
            self.stack.append(self.buffer.pop(0))
        elif transition == "LA":
            # Left-Arc : Top of stack is the head and second Top is the dependent & Remove the second top from stack & Keep the head
            # Insert in dependencies list ( head, dependent )
            self.dependencies.append((self.stack[-1], self.stack.pop(-2)))
        elif transition == "RA":
            # Right-Arc : Top of stack is the dependent and second Top is the head & Remove the first top from stack & Keep the head
            self.dependencies.append((self.stack[-2], self.stack.pop()))

        ### END YOUR CODE

    def parse(self, transitions):
        """Applies the provided transitions to this PartialParse

        @param transitions (list of str): The list of transitions in the order they should be applied

        @return dsependencies (list of string tuples): The list of dependencies produced when
                                                        parsing the sentence. Represented as a list of
                                                        tuples where each tuple is of the form (head, dependent).
        """
        for transition in transitions:
            self.parse_step(transition)
        return self.dependencies


def minibatch_parse(sentences, model, batch_size):
    """Parses a list of sentences in minibatches using a model.

    @param sentences (list of list of str): A list of sentences to be parsed
                                            (each sentence is a list of words and each word is of type string)
    @param model (ParserModel): The model that makes parsing decisions. It is assumed to have a function
                                model.predict(partial_parses) that takes in a list of PartialParses as input and
                                returns a list of transitions predicted for each parse. That is, after calling
                                    transitions = model.predict(partial_parses)
                                transitions[i] will be the next transition to apply to partial_parses[i].
    @param batch_size (int): The number of PartialParses to include in each minibatch


    @return dependencies (list of dependency lists): A list where each element is the dependencies
                                                    list for a parsed sentence. Ordering should be the
                                                    same as in sentences (i.e., dependencies[i] should
                                                    contain the parse for sentences[i]).
    """
    dependencies = []

    ### YOUR CODE HERE (~8-10 Lines)
    ### TODO:
    ###     Implement the minibatch parse algorithm as described in the pdf handout
    ###
    ###     Note: A shallow copy (as denoted in the PDF) can be made with the "=" sign in python, e.g.
    ###                 unfinished_parses = partial_parses[:].
    ###             Here `unfinished_parses` is a shallow copy of `partial_parses`.
    ###             In Python, a shallow copied list like `unfinished_parses` does not contain new instances
    ###             of the object stored in `partial_parses`. Rather both lists refer to the same objects.
    ###             In our case, `partial_parses` contains a list of partial parses. `unfinished_parses`
    ###             contains references to the same objects. Thus, you should NOT use the `del` operator
    ###             to remove objects from the `unfinished_parses` list. This will free the underlying memory that
    ###             is being accessed by `partial_parses` and may cause your code to crash.

    # Initialize partial parses as a list of PartialParses, one for each sentence in sentences
    partial_parses  = [None]*len(sentences)
    for i in range(len(sentences)):
        partial_parses[i] = PartialParse(sentences[i])

        
    unfinished_parses = partial_parses[:]
    while len(unfinished_parses) > 0:
        batch = unfinished_parses[:batch_size]
        transitions = model.predict(batch)
        for parse_, trans in zip(batch, transitions):
            parse_.parse_step(trans)
            if len(parse_.buffer) == 0 and len(parse_.stack) ==1:
                unfinished_parses.remove(parse_)

    dependencies = [p.dependencies for p in partial_parses]
    ### END YOUR CODE

    return dependencies


def test_step(name, transition, stack, buf, deps,
              ex_stack, ex_buf, ex_deps):
    """Tests that a single parse step returns the expected output"""
    pp = PartialParse([])
    pp.stack, pp.buffer, pp.dependencies = stack, buf, deps

    pp.parse_step(transition)
    stack, buf, deps = (tuple(pp.stack), tuple(pp.buffer), tuple(sorted(pp.dependencies)))
    assert stack == ex_stack, \
        "{:} test resulted in stack {:}, expected {:}".format(name, stack, ex_stack)
    assert buf == ex_buf, \
        "{:} test resulted in buffer {:}, expected {:}".format(name, buf, ex_buf)
    assert deps == ex_deps, \
        "{:} test resulted in dependency list {:}, expected {:}".format(name, deps, ex_deps)
    print("{:} test passed!".format(name))


def test_parse_step():
    """Simple tests for the PartialParse.parse_step function
    Warning: these are not exhaustive
    """
    test_step("SHIFT", "S", ["ROOT", "the"], ["cat", "sat"], [],
              ("ROOT", "the", "cat"), ("sat",), ())
    test_step("LEFT-ARC", "LA", ["ROOT", "the", "cat"], ["sat"], [],
              ("ROOT", "cat",), ("sat",), (("cat", "the"),))
    test_step("RIGHT-ARC", "RA", ["ROOT", "run", "fast"], [], [],
              ("ROOT", "run",), (), (("run", "fast"),))


def test_parse():
    """Simple tests for the PartialParse.parse function
    Warning: these are not exhaustive
    """
    sentence = ["parse", "this", "sentence"]
    dependencies = PartialParse(sentence).parse(["S", "S", "S", "LA", "RA", "RA"])
    dependencies = tuple(sorted(dependencies))
    expected = (('ROOT', 'parse'), ('parse', 'sentence'), ('sentence', 'this'))
    assert dependencies == expected,  \
        "parse test resulted in dependencies {:}, expected {:}".format(dependencies, expected)
    assert tuple(sentence) == ("parse", "this", "sentence"), \
        "parse test failed: the input sentence should not be modified"
    print("parse test passed!")


class DummyModel(object):
    """Dummy model for testing the minibatch_parse function
    """
    def __init__(self, mode = "unidirectional"):
        self.mode = mode

    def predict(self, partial_parses):
        if self.mode == "unidirectional":
            return self.unidirectional_predict(partial_parses)
        elif self.mode == "interleave":
            return self.interleave_predict(partial_parses)
        else:
            raise NotImplementedError()

    def unidirectional_predict(self, partial_parses):
        """First shifts everything onto the stack and then does exclusively right arcs if the first word of
        the sentence is "right", "left" if otherwise.
        """
        return [("RA" if pp.stack[1] is "right" else "LA") if len(pp.buffer) == 0 else "S"
                for pp in partial_parses]

    def interleave_predict(self, partial_parses):
        """First shifts everything onto the stack and then interleaves "right" and "left".
        """
        return [("RA" if len(pp.stack) % 2 == 0 else "LA") if len(pp.buffer) == 0 else "S"
                for pp in partial_parses]

def test_dependencies(name, deps, ex_deps):
    """Tests the provided dependencies match the expected dependencies"""
    deps = tuple(sorted(deps))
    assert deps == ex_deps, \
        "{:} test resulted in dependency list {:}, expected {:}".format(name, deps, ex_deps)


def test_minibatch_parse():
    """Simple tests for the minibatch_parse function
    Warning: these are not exhaustive
    """

    # Unidirectional arcs test
    sentences = [["right", "arcs", "only"],
                 ["right", "arcs", "only", "again"],
                 ["left", "arcs", "only"],
                 ["left", "arcs", "only", "again"]]
    deps = minibatch_parse(sentences, DummyModel(), 2)
    test_dependencies("minibatch_parse", deps[0],
                      (('ROOT', 'right'), ('arcs', 'only'), ('right', 'arcs')))
    test_dependencies("minibatch_parse", deps[1],
                      (('ROOT', 'right'), ('arcs', 'only'), ('only', 'again'), ('right', 'arcs')))
    test_dependencies("minibatch_parse", deps[2],
                      (('only', 'ROOT'), ('only', 'arcs'), ('only', 'left')))
    test_dependencies("minibatch_parse", deps[3],
                      (('again', 'ROOT'), ('again', 'arcs'), ('again', 'left'), ('again', 'only')))

    # Out-of-bound test
    sentences = [["right"]]
    deps = minibatch_parse(sentences, DummyModel(), 2)
    test_dependencies("minibatch_parse", deps[0], (('ROOT', 'right'),))

    # Mixed arcs test
    sentences = [["this", "is", "interleaving", "dependency", "test"]]
    deps = minibatch_parse(sentences, DummyModel(mode="interleave"), 1)
    test_dependencies("minibatch_parse", deps[0],
                      (('ROOT', 'is'), ('dependency', 'interleaving'),
                      ('dependency', 'test'), ('is', 'dependency'), ('is', 'this')))
    print("minibatch_parse test passed!")


'''if __name__ == '__main__':
    args = sys.argv
    if len(args) != 2:
        raise Exception("You did not provide a valid keyword. Either provide 'part_c' or 'part_d', when executing this script")
    elif args[1] == "part_c":
        test_parse_step()
        test_parse()
    elif args[1] == "part_d":
        test_minibatch_parse()
    else:
        raise Exception("You did not provide a valid keyword. Either provide 'part_c' or 'part_d', when executing this script")
'''


'if __name__ == \'__main__\':\n    args = sys.argv\n    if len(args) != 2:\n        raise Exception("You did not provide a valid keyword. Either provide \'part_c\' or \'part_d\', when executing this script")\n    elif args[1] == "part_c":\n        test_parse_step()\n        test_parse()\n    elif args[1] == "part_d":\n        test_minibatch_parse()\n    else:\n        raise Exception("You did not provide a valid keyword. Either provide \'part_c\' or \'part_d\', when executing this script")\n'

# (C)

In [0]:
test_parse_step()
test_parse()

SHIFT test passed!
LEFT-ARC test passed!
RIGHT-ARC test passed!
parse test passed!


In [0]:
!python parser_transitions.py part_c

SHIFT test passed!
LEFT-ARC test passed!
RIGHT-ARC test passed!
parse test passed!


# (D)


In [0]:
test_minibatch_parse()

minibatch_parse test passed!


# (E)

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

tt = nn.Parameter(nn.init.xavier_uniform_(torch.empty(2,5)))
tt

Parameter containing:
tensor([[-0.6069, -0.5430, -0.1276, -0.1830, -0.6857],
        [-0.4782,  0.0298,  0.7539, -0.2688, -0.8564]], requires_grad=True)

In [0]:
import numpy as np
x = torch.randn(3, 4)
indices = torch.tensor([0, 2])
torch.index_select(x, 0, indices)


tensor([[ 2.1358,  1.4621, -0.5189, -0.7111],
        [ 0.5240,  0.9194, -0.3916, -0.0295]])

In [0]:
!wget http://web.stanford.edu/class/cs224n/assignments/a3.zip

In [0]:
!unzip a3.zip

In [8]:
%cd student


/content/student


In [0]:
#Copied my code there
!python parser_model.py -e

Embedding_lookup sanity check passes!


In [0]:
!python parser_model.py -f

Forward sanity check passes!


In [0]:
!python run.py -d

INITIALIZING
Loading data...
took 1.68 seconds
Building parser...
took 0.03 seconds
Loading pretrained embeddings...
took 1.91 seconds
Vectorizing data...
took 0.05 seconds
Preprocessing training data...
took 1.09 seconds
took 0.00 seconds

TRAINING
Epoch 1 out of 10
100% 48/48 [01:14<00:00,  1.15s/it]
Average Train Loss: 0.5061016219357649
Evaluating on dev set
125250it [00:00, 16169177.47it/s]
- dev UAS: 55.78
New best dev UAS! Saving model.

Epoch 2 out of 10
100% 48/48 [01:15<00:00,  1.17s/it]
Average Train Loss: 0.26751851042111713
Evaluating on dev set
125250it [00:00, 17784507.80it/s]
- dev UAS: 59.72
New best dev UAS! Saving model.

Epoch 3 out of 10
100% 48/48 [01:13<00:00,  1.15s/it]
Average Train Loss: 0.2212474038824439
Evaluating on dev set
125250it [00:00, 17567434.99it/s]
- dev UAS: 62.83
New best dev UAS! Saving model.

Epoch 4 out of 10
100% 48/48 [01:12<00:00,  1.14s/it]
Average Train Loss: 0.18796728768696389
Evaluating on dev set
125250it [00:00, 18152611.47it/s]
- 

In [10]:
!python run.py

INITIALIZING
Loading data...
took 1.60 seconds
Building parser...
took 1.24 seconds
Loading pretrained embeddings...
took 2.14 seconds
Vectorizing data...
took 1.54 seconds
Preprocessing training data...
took 35.76 seconds
took 0.01 seconds

TRAINING
Epoch 1 out of 10
100% 1848/1848 [06:25<00:00,  4.88it/s]
Average Train Loss: 0.15452668404214698
Evaluating on dev set
1445850it [00:00, 54827763.51it/s]
- dev UAS: 83.83
New best dev UAS! Saving model.

Epoch 2 out of 10
100% 1848/1848 [06:26<00:00,  4.89it/s]
Average Train Loss: 0.09556062513905944
Evaluating on dev set
1445850it [00:00, 57528738.48it/s]
- dev UAS: 85.94
New best dev UAS! Saving model.

Epoch 3 out of 10
100% 1848/1848 [06:21<00:00,  4.77it/s]
Average Train Loss: 0.08250963118620765
Evaluating on dev set
1445850it [00:00, 56581367.98it/s]
- dev UAS: 86.76
New best dev UAS! Saving model.

Epoch 4 out of 10
100% 1848/1848 [06:19<00:00,  4.80it/s]
Average Train Loss: 0.074705859138207
Evaluating on dev set
1445850it [00:00

In [11]:
!python run.py -d

INITIALIZING
Loading data...
took 1.73 seconds
Building parser...
took 0.03 seconds
Loading pretrained embeddings...
took 1.97 seconds
Vectorizing data...
took 0.05 seconds
Preprocessing training data...
took 1.08 seconds
took 0.00 seconds

TRAINING
Epoch 1 out of 10
100% 48/48 [00:10<00:00,  4.70it/s]
Average Train Loss: 0.6258880458772182
Evaluating on dev set
125250it [00:00, 16115607.58it/s]
- dev UAS: 54.73
New best dev UAS! Saving model.

Epoch 2 out of 10
100% 48/48 [00:10<00:00,  4.66it/s]
Average Train Loss: 0.29806734931965667
Evaluating on dev set
125250it [00:00, 13111452.72it/s]
- dev UAS: 59.62
New best dev UAS! Saving model.

Epoch 3 out of 10
100% 48/48 [00:10<00:00,  4.70it/s]
Average Train Loss: 0.24989545562614998
Evaluating on dev set
125250it [00:00, 16402928.03it/s]
- dev UAS: 62.48
New best dev UAS! Saving model.

Epoch 4 out of 10
100% 48/48 [00:10<00:00,  4.69it/s]
Average Train Loss: 0.21980773533384004
Evaluating on dev set
125250it [00:00, 13717075.98it/s]
-